In [1]:
from locax.person_loader import load_persons
from locax.spread_analysis import SpreadAnalysis

persons = load_persons()

spread_analysis = SpreadAnalysis(persons)
spread_analysis.detect_crossings()

min_timestamp = persons[0].locations[0]['timestampMs'] / 1000
max_timestamp =  persons[0].locations[-1]['timestampMs'] / 1000

from ipywidgets import IntSlider, Output
from ipyleaflet import Map, Marker, Icon

reference_point = (49.6116, 6.1319)
map = Map(center=reference_point)

valid_spread_icon = Icon(icon_url='https://i.imgur.com/bpI2nha.png', icon_size=[12, 12], icon_anchor=[6,6])
default_icon = Icon(icon_url='https://i.imgur.com/TFvv9tV.png', icon_size=[12, 12], icon_anchor=[6,6])


for person in persons:
    relevant_location = person.get_location_at_timestamp(min_timestamp)
    marker = Marker(location=relevant_location, draggable=False, icon=default_icon)
    map.add_layer(marker)
    person.marker = marker

import os

min_timestamp, max_timestamp

timestamp_slider = IntSlider(min=min_timestamp, max=max_timestamp)

def on_slider_change(change_event):
    timestamp = change_event.new
    if isinstance(timestamp, dict):
        if 'value' not in timestamp:
            return
        timestamp = timestamp['value']

    for person in persons:
        location = person.get_location_at_timestamp(timestamp)
        person.marker.location = location
        
        if person.has_valid_spread_at_timestamp(timestamp):
            person.marker.icon = valid_spread_icon
    
timestamp_slider.observe(on_slider_change)

display(timestamp_slider, map)

IntSlider(value=1585800000, max=1585864780, min=1585800000)

Map(center=[49.6116, 6.1319], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…